In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
# Create an instance of the neural network
model = Net()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [6]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Print the training loss for each epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/10, Loss: 0.25327837467193604
Epoch 2/10, Loss: 0.3734153211116791
Epoch 3/10, Loss: 0.25435543060302734
Epoch 4/10, Loss: 0.33283886313438416
Epoch 5/10, Loss: 0.13344308733940125
Epoch 6/10, Loss: 0.06735716760158539
Epoch 7/10, Loss: 0.3119622468948364
Epoch 8/10, Loss: 0.06611382216215134
Epoch 9/10, Loss: 0.09190545976161957
Epoch 10/10, Loss: 0.037873003631830215


In [7]:
# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy}%')


Test Accuracy: 96.58%


In [8]:
model

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [9]:
model.fc1.weight

Parameter containing:
tensor([[-0.0283,  0.0152,  0.0158,  ...,  0.0014, -0.0042, -0.0196],
        [-0.0333, -0.0167,  0.0326,  ..., -0.0346, -0.0210, -0.0121],
        [-0.0372, -0.0150,  0.0331,  ..., -0.0179, -0.0187, -0.0016],
        ...,
        [-0.0279,  0.0319,  0.0095,  ...,  0.0096,  0.0053, -0.0107],
        [-0.0315, -0.0278, -0.0348,  ...,  0.0171,  0.0220,  0.0150],
        [ 0.0266,  0.0271, -0.0241,  ..., -0.0025,  0.0267, -0.0137]],
       requires_grad=True)

In [10]:
# Assuming 'model' is your PyTorch neural network model
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

fc1.weight tensor([[-0.0283,  0.0152,  0.0158,  ...,  0.0014, -0.0042, -0.0196],
        [-0.0333, -0.0167,  0.0326,  ..., -0.0346, -0.0210, -0.0121],
        [-0.0372, -0.0150,  0.0331,  ..., -0.0179, -0.0187, -0.0016],
        ...,
        [-0.0279,  0.0319,  0.0095,  ...,  0.0096,  0.0053, -0.0107],
        [-0.0315, -0.0278, -0.0348,  ...,  0.0171,  0.0220,  0.0150],
        [ 0.0266,  0.0271, -0.0241,  ..., -0.0025,  0.0267, -0.0137]])
fc1.bias tensor([-0.0006, -0.0208, -0.0046, -0.0122, -0.0029, -0.0098,  0.0033, -0.0086,
        -0.0294,  0.0044,  0.0240,  0.0359, -0.0122,  0.0314, -0.0074,  0.0158,
         0.0200,  0.0325, -0.0236,  0.0036, -0.0293,  0.0369,  0.0238,  0.0262,
         0.0189, -0.0262,  0.0124,  0.0302,  0.0362, -0.0273,  0.0185, -0.0154,
         0.0208,  0.0162, -0.0323,  0.0110, -0.0129,  0.0327, -0.0241,  0.0346,
         0.0035, -0.0098,  0.0263, -0.0251, -0.0347, -0.0057,  0.0254,  0.0304,
         0.0330,  0.0514,  0.0018, -0.0125, -0.0189,  0.0327,  0.0

In [11]:
torch.save(model.state_dict(), 'model.pth')

In [12]:
model_copy = Net()  # Assuming 'Net' is the class definition of your neural network
model_copy.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [13]:
weights_fc1 = model_copy.fc1.weight.data.clone()
biasess_fc1 = model_copy.fc1.bias.data.clone()
weights_fc2 = model_copy.fc2.weight.data.clone()
biasess_fc2 = model_copy.fc2.bias.data.clone()

In [14]:
print(weights_fc1.shape, weights_fc2.shape)
print(biasess_fc1.shape, biasess_fc2.shape)

torch.Size([128, 784]) torch.Size([64, 128])
torch.Size([128]) torch.Size([64])


In [15]:
model_copy.fc1.weight.data = weights_fc1
model_copy.fc1.bias.data = biasess_fc1
model_copy.fc2.weight.data = weights_fc2
model_copy.fc2.bias.data = biasess_fc2

In [27]:
weights_fc1.shape

torch.Size([128, 784])

In [32]:
weights_fc1[-1,-1]

tensor(-0.0137)

In [16]:
model_copy.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model_copy(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total

print(f'Test Accuracy model_copy: {accuracy}%')

correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total

print(f'Test Accuracy model: {accuracy}%')


Test Accuracy model_copy: 96.58%
Test Accuracy model: 96.58%


In [17]:
class Net_copy(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [18]:
# Create an instance of the neural network
model2 = Net()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.01)

In [19]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model2(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Print the training loss for each epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/10, Loss: 0.4088421165943146
Epoch 2/10, Loss: 0.11871203780174255
Epoch 3/10, Loss: 0.4515531659126282
Epoch 4/10, Loss: 0.5418092012405396
Epoch 5/10, Loss: 0.09428226202726364
Epoch 6/10, Loss: 0.14835457503795624
Epoch 7/10, Loss: 0.3167040944099426
Epoch 8/10, Loss: 0.017398031428456306
Epoch 9/10, Loss: 0.04062199592590332
Epoch 10/10, Loss: 0.16353219747543335


In [20]:
# Evaluate the model on the test set
model2.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model2(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy}%')


Test Accuracy: 96.17%
